# Advanced Deep Learning with Keras in Python

## The Keras Functional API


### Input layers

In [58]:
# Import Input from keras.layers
from keras.layers import Input

# Create an input layer of shape 1
input_tensor = Input(shape=(1,))

### Dense layers

In [59]:
# Load layers
from keras.layers import Input, Dense

# Input layer
input_tensor = Input(shape=(1,))

# Dense layer
output_layer = Dense(1)

# Connect the dense layer to the input_tensor
output_tensor = output_layer(input_tensor)

This network will take the input, apply a linear coefficient to it, and return the result.

### Output layers

In [60]:
# Load layers
from keras.layers import Input, Dense

# Input layer
input_tensor = Input(shape=(1,))

# Create a dense layer and connect the dense layer to the input_tensor in one step
# Note that we did this in 2 steps in the previous exercise, but are doing it in one step now
output_tensor = Dense(1)(input_tensor)

The output layer allows your model to make predictions.

### Build a model

In [61]:
# Input/dense/output layers
from keras.layers import Input, Dense
input_tensor = Input(shape=(1,))
output_tensor = Dense(1)(input_tensor)

# Build the model
from keras.models import Model
model = Model(input_tensor, output_tensor)

### Compile a model

This finalizes your model, freezes all its settings, and prepares it to meet some data!

In [62]:
# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

### Visualize a model

In [63]:
# Import the plotting function
from keras.utils import plot_model
import matplotlib.pyplot as plt

# Summarize the model
model.summary()

# # Plot the model
plot_model(model, to_file='model.png')

# # Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


### Fit the model to the tournament basketball data

In [64]:
import pandas as pd

games_tourney_train = pd.read_csv('basketball_data/games_tourney.csv')
games_tourney_train.head()

,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won
0,1985,288,73,0,-3,-9,41,50,0
1,1985,5929,73,0,4,6,61,55,1
2,1985,9884,73,0,5,-4,59,63,0
3,1985,73,288,0,3,9,50,41,1
4,1985,3920,410,0,1,-9,54,63,0


In [65]:
games_tourney_train.shape

(4234, 9)

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(games_tourney_train['seed_diff'], games_tourney_train['score_diff'])

In [67]:
# Now fit the model
model.fit(X_train, y_train,
          epochs=1,
          batch_size=128,
          validation_split=.10,
          verbose=True)

Train on 2857 samples, validate on 318 samples
Epoch 1/1
2857/2857 [==============================] - 0s 134us/step - loss: 9.6618 - val_loss: 10.2915


### Evaluate the model on a test set

In [68]:
# Evaluate the model on the test data
model.evaluate(X_test, y_test)

1059/1059 [==============================] - 0s 13us/step


9.887786717545435

## Two Input Networks Using Categorical Embeddings, Shared Layers, and Merge Layers

### Define team lookup

In [69]:
games_season = pd.read_csv('basketball_data/games_season.csv')
games_season.head()

,season,team_1,team_2,home,score_diff,score_1,score_2,won
0,1985,3745,6664,0,17,81,64,1
1,1985,126,7493,1,7,77,70,1
2,1985,288,3593,1,7,63,56,1
3,1985,1846,9881,1,16,70,54,1
4,1985,2675,10298,1,12,86,74,1


In [70]:
# Imports
from keras.layers import Embedding
from numpy import unique

# Count the unique number of teams
n_teams = unique(games_season['team_1']).shape[0]

# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

The embedding layer is a lot like a dictionary, but your model learns the values for each key.

### Define team model

In [71]:
# Imports
from keras.layers import Input, Embedding, Flatten
from keras.models import Model

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

### Defining two inputs

In [72]:
# Load the input layer from keras.layers
from keras.layers import Input

# Input layer for team 1
team_in_1 = Input((1,), name='Team-1-In')

# Separate input layer for team 2
team_in_2 = Input((1,), name='Team-2-In')

These two inputs will be used later for the shared layer.

### Lookup both inputs in the same model

In [73]:
# Lookup team 1 in the team strength model
team_1_strength = team_strength_model(team_in_1)

# Lookup team 2 in the team strength model
team_2_strength = team_strength_model(team_in_2)

Now your model knows how strong each team is.

### Output layer using shared layer

In [74]:
# Import the Subtract layer from keras
from keras.layers import Subtract

# Create a subtract layer using the inputs from the previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

This setup subracts the team strength ratings to determine a winner.

### Model using two inputs and one output

![](basketball_model_2.png)

In [75]:
# Imports
from keras.layers import Subtract
from keras.models import Model

# Subtraction layer from previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

# Create the model
model = Model([team_in_1, team_in_2], score_diff)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [76]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Team-1-In (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
Team-2-In (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
Team-Strength-Model (Model)     (None, 1)            10888       Team-1-In[0][0]                  
                                                                 Team-2-In[0][0]                  
__________________________________________________________________________________________________
subtract_4 (Subtract)           (None, 1)            0           Team-Strength-Model[1][0]        
          

### Fit the model to the regular season training data

In [77]:
# Get the team_1 column from the regular season data
input_1 = games_season['team_1']

# Get the team_2 column from the regular season data
input_2 = games_season['team_2']

# Fit the model to input 1 and 2, using score diff as a target
model.fit([input_1, input_2],
          games_season['score_diff'],
          epochs=1,
          batch_size=2048,
          validation_split=.1,
          verbose=True)

Train on 280960 samples, validate on 31218 samples
Epoch 1/1
280960/280960 [==============================] - 1s 2us/step - loss: 12.1204 - val_loss: 11.8380


Now our model has learned a strength rating for every team.

### Evaluate the model on the tournament test data

In [78]:
games_tourney = pd.read_csv('basketball_data/games_tourney.csv')
games_tourney.head()

,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won
0,1985,288,73,0,-3,-9,41,50,0
1,1985,5929,73,0,4,6,61,55,1
2,1985,9884,73,0,5,-4,59,63,0
3,1985,73,288,0,3,9,50,41,1
4,1985,3920,410,0,1,-9,54,63,0


In [79]:
# Get team_1 from the tournament data
input_1 = games_tourney['team_1']

# Get team_2 from the tournament data
input_2 = games_tourney['team_2']

# Evaluate the model using these inputs
model.evaluate([input_1, input_2], games_tourney['score_diff'])

4234/4234 [==============================] - 0s 21us/step


11.682201143588218

## Multiple Inputs: 3 Inputs (and Beyond!)

### Make an input layer for home vs. away

You know there is a well-documented home-team advantage in basketball, so you will add a new input to your model to capture this effect.



In [80]:
from keras.layers.merge import Concatenate

In [81]:
# Create an Input for each team
team_in_1 = Input(shape=(1,), name='Team-1-In')
team_in_2 = Input(shape=(1,), name='Team-2-In')

# Create an input for home vs away
home_in = Input(shape=(1,), name='Home-In')

# Lookup the team inputs in the team strength model
team_1_strength = team_strength_model(team_in_1)
team_2_strength = team_strength_model(team_in_2)

# Combine the team strengths with the home input using a Concatenate layer, then add a Dense layer
out = Concatenate()([team_1_strength, team_2_strength, home_in])
out = Dense(1)(out)

Now you have a model with 3 inputs!

### Make a model and compile it

In [82]:
# Import the model class
from keras.models import Model

# Make a Model
model = Model([team_in_1, team_in_2, home_in], out)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [83]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Team-1-In (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
Team-2-In (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
Team-Strength-Model (Model)     (None, 1)            10888       Team-1-In[0][0]                  
                                                                 Team-2-In[0][0]                  
__________________________________________________________________________________________________
Home-In (InputLayer)            (None, 1)            0                                            
__________

### Fit the model and evaluate

In [84]:
# Fit the model to the games_season dataset
model.fit([games_season['team_1'], games_season['team_2'], games_season['home']],
          games_season['score_diff'],
          epochs=1,
          verbose=True,
          validation_split=.1,
          batch_size=2048)

# Evaluate the model on the games_tourney dataset
model.evaluate([games_tourney['team_1'], games_tourney['team_2'], games_tourney['home']], games_tourney['score_diff'])

Train on 280960 samples, validate on 31218 samples
Epoch 1/1
4234/4234 [==============================] - 0s 21us/step


11.684669349573106

### Plotting models

In [ ]:
# Imports
import matplotlib.pyplot as plt
from keras.utils import plot_model

# Plot the model
plot_model(model, to_file='model.png')

# Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()

### Add the model predictions to the tournament data

In [86]:
# Predict
games_tourney['pred'] = model.predict([games_tourney['team_1'],games_tourney['team_2'],games_tourney['home']])

In [87]:
games_tourney.head()

,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won,pred
0,1985,288,73,0,-3,-9,41,50,0,0.157927
1,1985,5929,73,0,4,6,61,55,1,0.262091
2,1985,9884,73,0,5,-4,59,63,0,0.246523
3,1985,73,288,0,3,9,50,41,1,0.242940
4,1985,3920,410,0,1,-9,54,63,0,0.257307


In [88]:
games_tourney.score_diff.unique()

array([ -9,   6,  -4,   9,   3,  -2,  -1,   2,   5,   1, -13,   4,  -6,
       -25, -12, -18, -17,  16,  -8,  -7, -16,   8, -19,  -3,  11, -11,
        25, -24,  12,  23,  -5,   7,  19, -23,  13,  17,  24,  18,  10,
        14, -10,  32,  49,  20, -21, -28, -15, -22, -20, -14,  15, -32,
        21,  28, -49,  22,  34, -34, -31,  31, -40, -29,  40,  26,  27,
       -26, -27,  29,  43, -37, -43,  37,  30, -35,  35, -30, -41,  41,
       -38,  38,  39,  33, -39, -33, -44, -45,  45,  44,  36, -46,  46,
       -36, -58,  58, -42,  42, -47,  47,  56, -56])

Now you can try building a model for the tournament data based on your regular season predictions

### Create an input layer with multiple columns

In [89]:
# Create an input layer with 3 columns
input_tensor = Input((3,))

# Pass it to a Dense layer with 1 unit
output_tensor = Dense(1)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

### Fit the model

In [97]:
games_tourney_train = games_tourney.query('season < 2010')

In [98]:
games_tourney_train.shape

(3168, 10)

In [99]:
# Fit the model
model.fit(games_tourney_train[['home', 'seed_diff', 'pred']],
          games_tourney_train['score_diff'],
          epochs=1,
          verbose=True)

Epoch 1/1
3168/3168 [==============================] - 0s 37us/step - loss: 12.6005


### Evaluate the model

In [100]:
games_tourney_test = games_tourney.query('season >= 2010')

In [101]:
games_tourney_test.shape

(1066, 10)

### Evaluate the model

In [102]:
# Evaluate the model on the games_tourney_test dataset
model.evaluate(games_tourney_test[['home', 'seed_diff', 'pred']], 
               games_tourney_test['score_diff'])

1066/1066 [==============================] - 0s 16us/step


11.704881755764443

## Multiple Outputs

### Simple two-output model

"multiple target regression": one model making more than one prediction.

In [103]:
# Define the input
input_tensor = Input((2,))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

### Fit a model with two outputs

 this model will predict the scores of both teams.

In [105]:
games_tourney_train.shape

(3168, 10)

In [107]:
# Fit the model
model.fit(games_tourney_train[['seed_diff', 'pred']],
  		  games_tourney_train[['score_1', 'score_2']],
  		  verbose=False,
  		  epochs=1000,
  		  batch_size=64)

In [116]:
import numpy as np

In [117]:
np.mean(model.history.history['loss'])

35.43869778077289

### Inspect the model (I)

The input layer will have 4 weights: 2 for each input times 2 for each output.

The output layer will have 2 weights, one for each output.

In [118]:
# Print the model's weights
print(model.get_weights())

# Print the column means of the training data
print(games_tourney_train.mean())

[array([[ 0.54387677, -0.69338655],
       [53.29618   , 54.129517  ]], dtype=float32), array([53.91616 , 53.871307], dtype=float32)]
season        1997.045455
team_1        5546.025568
team_2        5546.025568
home             0.000000
seed_diff        0.000000
score_diff       0.000000
score_1         71.912247
score_2         71.912247
won              0.500000
pred             0.236631
dtype: float64


Did you notice that both output weights are about ~53? This is because, on average, a team will score about 53 points in the tournament.

### Evaluate the model

In [121]:
# Evaluate the model on the tournament test data
model.evaluate(games_tourney_test[['seed_diff', 'pred']],games_tourney_test[['score_1', 'score_2']])

1066/1066 [==============================] - 0s 127us/step


8.80074873561036

### Classification and regression in one model

In [122]:
# Create an input layer with 2 columns
input_tensor = Input((2,))

# Create the first output
output_tensor_1 = Dense(1, activation='linear', use_bias=False)(input_tensor)

# Create the second output (use the first output as input here)
output_tensor_2 = Dense(1, activation='sigmoid', use_bias=False)(output_tensor_1)

# Create a model with 2 outputs
model = Model(input_tensor, [output_tensor_1, output_tensor_2])

### Compile and fit the model

In [123]:
# Import the Adam optimizer
from keras.optimizers import Adam

# Compile the model with 2 losses and the Adam optimzer with a higher learning rate
model.compile(loss=['mean_absolute_error', 'binary_crossentropy'], optimizer=Adam(lr=0.01))

# Fit the model to the tournament training data, with 2 inputs and 2 outputs
model.fit(games_tourney_train[['seed_diff', 'pred']],
          [games_tourney_train[['score_diff']], games_tourney_train[['won']]],
          epochs=10,
          verbose=True,
          batch_size=16384)

Epoch 1/10
3168/3168 [==============================] - 0s 138us/step - loss: 10.7605 - dense_12_loss: 9.1626 - dense_13_loss: 1.5978
Epoch 2/10
3168/3168 [==============================] - 0s 1us/step - loss: 10.7327 - dense_12_loss: 9.1601 - dense_13_loss: 1.5726
Epoch 3/10
3168/3168 [==============================] - 0s 1us/step - loss: 10.7053 - dense_12_loss: 9.1579 - dense_13_loss: 1.5474
Epoch 4/10
3168/3168 [==============================] - 0s 1us/step - loss: 10.6788 - dense_12_loss: 9.1561 - dense_13_loss: 1.5227
Epoch 5/10
3168/3168 [==============================] - 0s 1us/step - loss: 10.6523 - dense_12_loss: 9.1545 - dense_13_loss: 1.4978
Epoch 6/10
3168/3168 [==============================] - 0s 1us/step - loss: 10.6264 - dense_12_loss: 9.1531 - dense_13_loss: 1.4733
Epoch 7/10
3168/3168 [==============================] - 0s 1us/step - loss: 10.6003 - dense_12_loss: 9.1520 - dense_13_loss: 1.4483
Epoch 8/10
3168/3168 [==============================] - 0s 1us/step - loss

### Inspect the model (II)

In [124]:
# Print the model weights
print(model.get_weights())

# Print the training data means
print(games_tourney_train.mean())

[array([[1.2275231 ],
       [0.21650095]], dtype=float32), array([[0.8894083]], dtype=float32)]
season        1997.045455
team_1        5546.025568
team_2        5546.025568
home             0.000000
seed_diff        0.000000
score_diff       0.000000
score_1         71.912247
score_2         71.912247
won              0.500000
pred             0.236631
dtype: float64


In [125]:
# Import the sigmoid function from scipy
from scipy.special import expit as sigmoid

# Weight from the model
weight = 0.14

# Print the approximate win probability predicted close game
print(sigmoid(1 * weight))

# Print the approximate win probability predicted blowout game
print(sigmoid(10 * weight))

0.5349429451582145
0.8021838885585818


So `sigmoid(1 * 0.14)` is `0.53`, which represents a pretty close game and `sigmoid(10 * 0.14)` is `0.80`, which represents a pretty likely win. In other words, if the model predicts a win of 1 point, it is less sure of the win than if it predicts 10 points. Who says neural networks are black boxes?

### Evaluate on new data with two metrics

Keras will return 3 numbers: 

the first number will be the sum of both the loss functions, 

the next 2 numbers will be the loss functions you used when defining the model.

In [126]:
# Evaluate the model on new data
model.evaluate(games_tourney_test[['seed_diff', 'pred']],
               [games_tourney_test[['score_diff']], games_tourney_test[['won']]])

1066/1066 [==============================] - 0s 149us/step


[10.716846509006636, 9.135157445581948, 1.5816891162077884]

model plays a role as a regressor and a good classifier